In [1]:
import sys
import dask
from datetime import datetime
import numpy as np
import xarray as xr

In [2]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:61550")
client

<Client: 'tcp://127.0.0.1:61550' processes=4 threads=8, memory=16.00 GiB>

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2
sys.path.insert(0, '../')
from data import erddap, local

# Remote ERDDAP server [(link)](https://data.pmel.noaa.gov/generic/erddap/tabledap/gdp_hourly_velocities.html)

## Unique drifter

In [5]:
# get drifter from errdap
e = erddap()
e.retrieve_drifter(10050130)
e.print_constraints()

All variables in this dataset:
None

Constraints of this dataset:
{
 "ID=": "10050130"
}


In [6]:
%%time
ds = e.to_xarray()
ds

CPU times: user 49 ms, sys: 10.4 ms, total: 59.4 ms
Wall time: 838 ms


<xarray.Dataset>
Dimensions:                (trajectory: 1, obs: 3592)
Coordinates:
    longitude              (obs) float32 ...
    latitude               (obs) float32 ...
    time                   (obs) datetime64[ns] ...
Dimensions without coordinates: trajectory, obs
Data variables: (12/47)
    ID                     (trajectory) object ...
    rowSize                (trajectory) int32 ...
    location_type          (obs) object ...
    WMO                    (obs) float64 ...
    expno                  (obs) float64 ...
    deploy_date            (obs) datetime64[ns] ...
    ...                     ...
    DrogueBallast          (obs) object ...
    DragAreaAboveDrogue    (obs) object ...
    DragAreaOfDrogue       (obs) object ...
    DragAreaRatio          (obs) object ...
    DrogueCenterDepth      (obs) object ...
    DrogueDetectSensor     (obs) object ...
Attributes: (12/51)
    acknowledgement:            Elipot et al. (2016). Global Drifter Program ...
    cdm_data_type:              Trajectory
    cdm_trajectory_variables:   ID
    comment:                    Global Drifter Program hourly data
    contributor_name:           NOAA Global Drifter Program
    contributor_role:           Data Acquisition Center
    ...                         ...
    subsetVariables:            ID, location_type, WMO, expno
    summary:                    Global Drifter Program hourly data
    time_coverage_end:          2012-07-14T18:00:00Z
    time_coverage_start:        2012-02-15T14:00:00Z
    title:                      Global Drifter Program hourly drifting buoy c...
    Westernmost_Easting:        103.261

## list of drifters

## region

In [10]:
e2 = erddap()

lon = [-98, -78]
lat = [18, 31]
day0 = "%sZ" % datetime(2015,1,1).isoformat()
day1 = "%sZ" % datetime(2020,12,31).isoformat()
time = [day0, day1]

e2.retrieve_region(lon, lat, time)

e2.print_constraints()

All variables in this dataset:
None

Constraints of this dataset:
{
 "longitude>=": -98,
 "longitude<=": -78,
 "latitude>=": 18,
 "latitude<=": 31,
 "time>=": "2015-01-01T00:00:00Z",
 "time<=": "2020-12-31T00:00:00Z"
}


In [11]:
%%time
ds2 = e2.to_xarray()
ds2

CPU times: user 455 ms, sys: 95 ms, total: 550 ms
Wall time: 7.34 s


<xarray.Dataset>
Dimensions:                (trajectory: 202, obs: 316142)
Coordinates:
    longitude              (obs) float32 ...
    latitude               (obs) float32 ...
    time                   (obs) datetime64[ns] ...
Dimensions without coordinates: trajectory, obs
Data variables: (12/47)
    ID                     (trajectory) object ...
    rowSize                (trajectory) int32 ...
    location_type          (obs) object ...
    WMO                    (obs) float64 ...
    expno                  (obs) float64 ...
    deploy_date            (obs) datetime64[ns] ...
    ...                     ...
    DrogueBallast          (obs) object ...
    DragAreaAboveDrogue    (obs) object ...
    DragAreaOfDrogue       (obs) object ...
    DragAreaRatio          (obs) object ...
    DrogueCenterDepth      (obs) object ...
    DrogueDetectSensor     (obs) object ...
Attributes: (12/51)
    acknowledgement:            Elipot et al. (2016). Global Drifter Program ...
    cdm_data_type:              Trajectory
    cdm_trajectory_variables:   ID
    comment:                    Global Drifter Program hourly data
    contributor_name:           NOAA Global Drifter Program
    contributor_role:           Data Acquisition Center
    ...                         ...
    subsetVariables:            ID, location_type, WMO, expno
    summary:                    Global Drifter Program hourly data
    time_coverage_end:          2020-03-31T23:00:00Z
    time_coverage_start:        2015-01-01T00:00:00Z
    title:                      Global Drifter Program hourly drifting buoy c...
    Westernmost_Easting:        -97.76723

# Local

In [12]:
path_clouddrift = '/Users/pmiron/OneDrive - Florida State University/projects/clouddrift/data/process/gdp_1.04c.nc'
#path_clouddrift = '/Users/pmiron/OneDrive - Florida State University/projects/clouddrift/data/process/gdp_1.04c'

In [13]:
%%time
l = local(path_clouddrift)

CPU times: user 69.3 ms, sys: 56.3 ms, total: 126 ms
Wall time: 146 ms


In [15]:
l.ds

<xarray.Dataset>
Dimensions:                 (traj: 17324, obs: 165754333)
Coordinates:
    longitude               (obs) float32 dask.array<chunksize=(10000000,), meta=np.ndarray>
    latitude                (obs) float32 dask.array<chunksize=(10000000,), meta=np.ndarray>
    time                    (obs) datetime64[ns] dask.array<chunksize=(10000000,), meta=np.ndarray>
    ids                     (obs) uint32 dask.array<chunksize=(10000000,), meta=np.ndarray>
Dimensions without coordinates: traj, obs
Data variables: (12/27)
    id                      (traj) uint32 dask.array<chunksize=(17324,), meta=np.ndarray>
    count                   (traj) int64 dask.array<chunksize=(17324,), meta=np.ndarray>
    wmo                     (traj) uint32 dask.array<chunksize=(17324,), meta=np.ndarray>
    expno                   (traj) uint32 dask.array<chunksize=(17324,), meta=np.ndarray>
    deploy_date             (traj) datetime64[ns] dask.array<chunksize=(17324,), meta=np.ndarray>
    deploy_lon              (traj) float32 dask.array<chunksize=(17324,), meta=np.ndarray>
    ...                      ...
    vn                      (obs) float32 dask.array<chunksize=(10000000,), meta=np.ndarray>
    err_lat                 (obs) float32 dask.array<chunksize=(10000000,), meta=np.ndarray>
    err_lon                 (obs) float32 dask.array<chunksize=(10000000,), meta=np.ndarray>
    err_ve                  (obs) float32 dask.array<chunksize=(10000000,), meta=np.ndarray>
    err_vn                  (obs) float32 dask.array<chunksize=(10000000,), meta=np.ndarray>
    gap                     (obs) timedelta64[ns] dask.array<chunksize=(10000000,), meta=np.ndarray>
Attributes: (12/17)
    title:             Global Drifter Program hourly drifting buoy collection
    id:                Global Drifter Program ID 13555
    history:           Version 1.04.  Metadata from dirall.dat and deplog.dat
    Conventions:       CF-1.6
    date_created:      2021-11-02T16:43:38.924905
    publisher_name:    GDP Drifter DAC
    ...                ...
    contributor_name:  NOAA Global Drifter Program
    contributor_role:  Data Acquisition Center
    institution:       NOAA Atlantic Oceanographic and Meteorological Laboratory
    acknowledgement:   Elipot et al. (2016). Global Drifter Program quality-c...
    doi:               10.1002/2016JC011716TBA
    summary:           Global Drifter Program hourly data

## unique drifter

In [16]:
%%time
ds = l.retrieve_drifter(10050130)

CPU times: user 24.4 ms, sys: 4.34 ms, total: 28.8 ms
Wall time: 135 ms


In [17]:
ds

<xarray.Dataset>
Dimensions:                 (traj: 1, obs: 3592)
Coordinates:
    longitude               (obs) float32 110.1 110.0 110.0 ... 108.1 108.1
    latitude                (obs) float32 -21.04 -21.06 -21.07 ... -23.15 -23.15
    time                    (obs) datetime64[ns] 2012-02-16T14:00:00 ... 2012...
    ids                     (obs) uint32 12287 12287 12287 ... 12287 12287 12287
Dimensions without coordinates: traj, obs
Data variables: (12/27)
    id                      (traj) uint32 10050130
    count                   (traj) int64 3592
    wmo                     (traj) uint32 5600935
    expno                   (traj) uint32 2222
    deploy_date             (traj) datetime64[ns] 2012-02-16
    deploy_lon              (traj) float32 110.2
    ...                      ...
    vn                      (obs) float32 -0.5712 -0.5609 ... 0.0964 0.0723
    err_lat                 (obs) float32 65.0 46.0 55.0 42.0 ... 55.0 50.0 83.0
    err_lon                 (obs) float32 70.0 49.0 59.0 45.0 ... 60.0 54.0 90.0
    err_ve                  (obs) float32 0.0089 0.0061 0.007 ... 0.0093 0.0165
    err_vn                  (obs) float32 0.0089 0.0061 0.007 ... 0.0093 0.0165
    gap                     (obs) timedelta64[ns] 02:29:46 02:29:46 ... 00:00:00
Attributes: (12/17)
    title:             Global Drifter Program hourly drifting buoy collection
    id:                Global Drifter Program ID 13555
    history:           Version 1.04.  Metadata from dirall.dat and deplog.dat
    Conventions:       CF-1.6
    date_created:      2021-11-02T16:43:38.924905
    publisher_name:    GDP Drifter DAC
    ...                ...
    contributor_name:  NOAA Global Drifter Program
    contributor_role:  Data Acquisition Center
    institution:       NOAA Atlantic Oceanographic and Meteorological Laboratory
    acknowledgement:   Elipot et al. (2016). Global Drifter Program quality-c...
    doi:               10.1002/2016JC011716TBA
    summary:           Global Drifter Program hourly data

## list of drifters

In [18]:
%%time
ds = l.retrieve_drifter([10050130, 10051120])

CPU times: user 694 ms, sys: 1.09 s, total: 1.78 s
Wall time: 2.06 s


In [19]:
ds

<xarray.Dataset>
Dimensions:                 (traj: 2, obs: 7861)
Coordinates:
    longitude               (obs) float32 110.1 110.0 110.0 ... 138.5 138.5
    latitude                (obs) float32 -21.04 -21.06 -21.07 ... -56.69 -56.68
    time                    (obs) datetime64[ns] 2012-02-16T14:00:00 ... 2012...
    ids                     (obs) uint32 12287 12287 12287 ... 12288 12288 12288
Dimensions without coordinates: traj, obs
Data variables: (12/27)
    id                      (traj) uint32 10050130 10051120
    count                   (traj) int64 3592 4269
    wmo                     (traj) uint32 5600935 5600524
    expno                   (traj) uint32 2222 2222
    deploy_date             (traj) datetime64[ns] 2012-02-16 2011-12-06
    deploy_lon              (traj) float32 110.2 110.1
    ...                      ...
    vn                      (obs) float32 -0.5712 -0.5609 ... 0.4091 0.4109
    err_lat                 (obs) float32 65.0 46.0 55.0 42.0 ... 57.0 46.0 55.0
    err_lon                 (obs) float32 70.0 49.0 59.0 ... 104.0 84.0 99.0
    err_ve                  (obs) float32 0.0089 0.0061 0.007 ... 0.0046 0.0055
    err_vn                  (obs) float32 0.0089 0.0061 0.007 ... 0.0046 0.0055
    gap                     (obs) timedelta64[ns] 02:29:46 02:29:46 ... 03:00:00
Attributes: (12/17)
    title:             Global Drifter Program hourly drifting buoy collection
    id:                Global Drifter Program ID 13555
    history:           Version 1.04.  Metadata from dirall.dat and deplog.dat
    Conventions:       CF-1.6
    date_created:      2021-11-02T16:43:38.924905
    publisher_name:    GDP Drifter DAC
    ...                ...
    contributor_name:  NOAA Global Drifter Program
    contributor_role:  Data Acquisition Center
    institution:       NOAA Atlantic Oceanographic and Meteorological Laboratory
    acknowledgement:   Elipot et al. (2016). Global Drifter Program quality-c...
    doi:               10.1002/2016JC011716TBA
    summary:           Global Drifter Program hourly data

In [20]:
%%time
idx_to_retrieve = l.ds['id'][np.random.randint(0, l.number_traj, 50)].values
ds = l.retrieve_drifter(idx_to_retrieve)

CPU times: user 1.35 s, sys: 1.18 s, total: 2.53 s
Wall time: 5.92 s


In [21]:
ds

<xarray.Dataset>
Dimensions:                 (traj: 50, obs: 409131)
Coordinates:
    longitude               (obs) float32 -2.032 -2.03 -2.027 ... -25.22 -25.2
    latitude                (obs) float32 -9.01 -9.01 -9.01 ... 37.01 37.01 37.0
    time                    (obs) datetime64[ns] 2015-08-29T18:00:00 ... 2010...
    ids                     (obs) uint32 13450 13450 13450 ... 5700 5700 5700
Dimensions without coordinates: traj, obs
Data variables: (12/27)
    id                      (traj) uint32 62958430 59322 114838 ... 139489 83395
    count                   (traj) int64 6067 1067 7026 9656 ... 5390 21043 9058
    wmo                     (traj) uint32 1500688 5100642 ... 4300550 4400845
    expno                   (traj) uint32 2222 7325 5325 6325 ... 6129 6129 6325
    deploy_date             (traj) datetime64[ns] 2015-08-29 ... 2009-03-12
    deploy_lon              (traj) float32 -2.0 -158.6 -55.07 ... -111.0 -63.02
    ...                      ...
    vn                      (obs) float32 0.0123 -0.0001 ... -0.0042 -0.0551
    err_lat                 (obs) float32 85.0 55.0 37.0 ... 0.00064 0.00336
    err_lon                 (obs) float32 86.0 55.0 37.0 ... 0.00173 0.00256
    err_ve                  (obs) float32 0.0369 0.0185 0.0137 ... 0.099 0.0677
    err_vn                  (obs) float32 0.0369 0.0185 0.0137 ... 0.0149 0.0731
    gap                     (obs) timedelta64[ns] 00:00:00 01:00:29 ... 01:12:00
Attributes: (12/17)
    title:             Global Drifter Program hourly drifting buoy collection
    id:                Global Drifter Program ID 13555
    history:           Version 1.04.  Metadata from dirall.dat and deplog.dat
    Conventions:       CF-1.6
    date_created:      2021-11-02T16:43:38.924905
    publisher_name:    GDP Drifter DAC
    ...                ...
    contributor_name:  NOAA Global Drifter Program
    contributor_role:  Data Acquisition Center
    institution:       NOAA Atlantic Oceanographic and Meteorological Laboratory
    acknowledgement:   Elipot et al. (2016). Global Drifter Program quality-c...
    doi:               10.1002/2016JC011716TBA
    summary:           Global Drifter Program hourly data

## region

In [24]:
%%time

lon = [-98, -78]
lat = [18, 31]
day0 = "%sZ" % datetime(2015,1,1).isoformat()
day1 = "%sZ" % datetime(2020,12,31).isoformat()
time = [day0, day1]
ds2 = l.retrieve_region(lon, lat, time)
ds2

CPU times: user 2.28 s, sys: 1.96 s, total: 4.24 s
Wall time: 7.75 s


<xarray.Dataset>
Dimensions:                 (traj: 202, obs: 316165)
Coordinates:
    longitude               (obs) float32 -87.51 -87.48 -87.44 ... -78.84 -78.85
    latitude                (obs) float32 29.42 29.42 29.41 ... 30.99 30.99
    time                    (obs) datetime64[ns] 2016-01-23T15:00:00 ... 2019...
    ids                     (obs) uint32 10083 10083 10083 ... 17193 17193 17193
Dimensions without coordinates: traj, obs
Data variables: (12/27)
    id                      (traj) uint32 102539 102541 ... 68054710 68055200
    count                   (traj) int64 1684 240 93 1460 ... 118 557 173 133
    wmo                     (traj) uint32 4200514 4300563 ... 4101695 4101694
    expno                   (traj) uint32 5325 5325 5325 ... 21312 21312 21312
    deploy_date             (traj) datetime64[ns] 2016-01-22 ... 2019-12-15
    deploy_lon              (traj) float32 -87.67 -83.68 ... -79.38 -79.68
    ...                      ...
    vn                      (obs) float32 -0.2091 -0.2115 ... 0.1908 0.2182
    err_lat                 (obs) float32 0.00158 0.001 0.0026 ... 37.0 38.0
    err_lon                 (obs) float32 0.00314 0.00277 0.00362 ... 44.0 44.0
    err_ve                  (obs) float32 0.0378 0.0409 0.0383 ... 0.0046 0.0046
    err_vn                  (obs) float32 0.0279 0.0278 0.034 ... 0.0046 0.0046
    gap                     (obs) timedelta64[ns] 01:42:14 01:36:29 ... 03:00:00
Attributes: (12/17)
    title:             Global Drifter Program hourly drifting buoy collection
    id:                Global Drifter Program ID 13555
    history:           Version 1.04.  Metadata from dirall.dat and deplog.dat
    Conventions:       CF-1.6
    date_created:      2021-11-02T16:43:38.924905
    publisher_name:    GDP Drifter DAC
    ...                ...
    contributor_name:  NOAA Global Drifter Program
    contributor_role:  Data Acquisition Center
    institution:       NOAA Atlantic Oceanographic and Meteorological Laboratory
    acknowledgement:   Elipot et al. (2016). Global Drifter Program quality-c...
    doi:               10.1002/2016JC011716TBA
    summary:           Global Drifter Program hourly data

In [26]:
ds2.mean()

<xarray.Dataset>
Dimensions:                 ()
Data variables: (12/23)
    id                      float64 3.709e+07
    count                   float64 1.565e+03
    wmo                     float64 4.074e+06
    expno                   float64 1.358e+04
    deploy_lon              float32 -75.13
    deploy_lat              float32 23.96
    ...                      ...
    vn                      float32 0.04583
    err_lat                 float32 22.46
    err_lon                 float32 25.12
    err_ve                  float32 -1.82
    err_vn                  float32 -1.829
    gap                     timedelta64[ns] 01:24:42.272114244

## groupby and map()

In [27]:
%%time
grouped_traj = l.ds.groupby('ids')

CPU times: user 24.4 s, sys: 1.76 s, total: 26.2 s
Wall time: 28 s


In [28]:
grouped_traj

DatasetGroupBy, grouped over 'ids'
17324 groups with labels 0, 1, 2, 3, ..., 17321, 17322, 17323.

In [30]:
grouped_traj[1]

<xarray.Dataset>
Dimensions:                 (traj: 17324, obs: 13911)
Coordinates:
    longitude               (obs) float32 dask.array<chunksize=(13911,), meta=np.ndarray>
    latitude                (obs) float32 dask.array<chunksize=(13911,), meta=np.ndarray>
    time                    (obs) datetime64[ns] dask.array<chunksize=(13911,), meta=np.ndarray>
    ids                     (obs) uint32 dask.array<chunksize=(13911,), meta=np.ndarray>
Dimensions without coordinates: traj, obs
Data variables: (12/27)
    id                      (traj) uint32 dask.array<chunksize=(17324,), meta=np.ndarray>
    count                   (traj) int64 dask.array<chunksize=(17324,), meta=np.ndarray>
    wmo                     (traj) uint32 dask.array<chunksize=(17324,), meta=np.ndarray>
    expno                   (traj) uint32 dask.array<chunksize=(17324,), meta=np.ndarray>
    deploy_date             (traj) datetime64[ns] dask.array<chunksize=(17324,), meta=np.ndarray>
    deploy_lon              (traj) float32 dask.array<chunksize=(17324,), meta=np.ndarray>
    ...                      ...
    vn                      (obs) float32 dask.array<chunksize=(13911,), meta=np.ndarray>
    err_lat                 (obs) float32 dask.array<chunksize=(13911,), meta=np.ndarray>
    err_lon                 (obs) float32 dask.array<chunksize=(13911,), meta=np.ndarray>
    err_ve                  (obs) float32 dask.array<chunksize=(13911,), meta=np.ndarray>
    err_vn                  (obs) float32 dask.array<chunksize=(13911,), meta=np.ndarray>
    gap                     (obs) timedelta64[ns] dask.array<chunksize=(13911,), meta=np.ndarray>
Attributes: (12/17)
    title:             Global Drifter Program hourly drifting buoy collection
    id:                Global Drifter Program ID 13555
    history:           Version 1.04.  Metadata from dirall.dat and deplog.dat
    Conventions:       CF-1.6
    date_created:      2021-11-02T16:43:38.924905
    publisher_name:    GDP Drifter DAC
    ...                ...
    contributor_name:  NOAA Global Drifter Program
    contributor_role:  Data Acquisition Center
    institution:       NOAA Atlantic Oceanographic and Meteorological Laboratory
    acknowledgement:   Elipot et al. (2016). Global Drifter Program quality-c...
    doi:               10.1002/2016JC011716TBA
    summary:           Global Drifter Program hourly data

In [46]:
grouped_traj[100].isel(traj=[100])

<xarray.Dataset>
Dimensions:                 (traj: 1, obs: 8659)
Coordinates:
    longitude               (obs) float32 dask.array<chunksize=(8659,), meta=np.ndarray>
    latitude                (obs) float32 dask.array<chunksize=(8659,), meta=np.ndarray>
    time                    (obs) datetime64[ns] dask.array<chunksize=(8659,), meta=np.ndarray>
    ids                     (obs) uint32 dask.array<chunksize=(8659,), meta=np.ndarray>
Dimensions without coordinates: traj, obs
Data variables: (12/27)
    id                      (traj) uint32 dask.array<chunksize=(1,), meta=np.ndarray>
    count                   (traj) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    wmo                     (traj) uint32 dask.array<chunksize=(1,), meta=np.ndarray>
    expno                   (traj) uint32 dask.array<chunksize=(1,), meta=np.ndarray>
    deploy_date             (traj) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    deploy_lon              (traj) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    ...                      ...
    vn                      (obs) float32 dask.array<chunksize=(8659,), meta=np.ndarray>
    err_lat                 (obs) float32 dask.array<chunksize=(8659,), meta=np.ndarray>
    err_lon                 (obs) float32 dask.array<chunksize=(8659,), meta=np.ndarray>
    err_ve                  (obs) float32 dask.array<chunksize=(8659,), meta=np.ndarray>
    err_vn                  (obs) float32 dask.array<chunksize=(8659,), meta=np.ndarray>
    gap                     (obs) timedelta64[ns] dask.array<chunksize=(8659,), meta=np.ndarray>
Attributes: (12/17)
    title:             Global Drifter Program hourly drifting buoy collection
    id:                Global Drifter Program ID 13555
    history:           Version 1.04.  Metadata from dirall.dat and deplog.dat
    Conventions:       CF-1.6
    date_created:      2021-11-02T16:43:38.924905
    publisher_name:    GDP Drifter DAC
    ...                ...
    contributor_name:  NOAA Global Drifter Program
    contributor_role:  Data Acquisition Center
    institution:       NOAA Atlantic Oceanographic and Meteorological Laboratory
    acknowledgement:   Elipot et al. (2016). Global Drifter Program quality-c...
    doi:               10.1002/2016JC011716TBA
    summary:           Global Drifter Program hourly data

# manually apply function at each trajectory

In [21]:
def distance(i):
    # get first and last point
    ds_subset = l.ds.isel(traj=[i], obs=slice(l.traj_idx[i], l.traj_idx[i + 1]))
    lon1 = ds_subset.longitude[0]
    lat1 = ds_subset.latitude[0]
    lon2 = ds_subset.longitude[-1]
    lat2 = ds_subset.latitude[-1]
    
    # convert to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # haversine
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [ ]:
%%time
a = distance(10)

In [32]:
a.compute()

KeyboardInterrupt: 

In [ ]:
import dask.bag as db
b = db.from_sequence(range(10), npartitions=10).map(distance)

In [ ]:
b

In [ ]:
%%time
d = b.compute()

In [ ]:
d

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(d)